En este notebook se tratará la instalación del dataset a usar por la herramienta. Este dataset lo proporciona _SoccerNet_ y contiene más de 3.000 _multi-view videos_ para entrenamiento, validación y prueba, además provee 273 acciones para el desafío.

Comenzamos instalamos la librería __SoccerNet__, que nos proporcionará las herramientas necesarias para la descarga del dataset.

In [7]:
pip install SoccerNet --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Usaremos la herramienta _SoccerNetDownloader_ para descargar el dataset. Para ello declararamos la ruta donde queremos que se almacene el dataset. Para la descarga del dataset es necesario adquirir una licencia de uso de este a través del siguiente enlace: https://docs.google.com/forms/d/e/1FAIpQLSfYFqjZNm4IgwGnyJXDPk2Ko_lZcbVtYX73w5lf6din5nxfmA/viewform

In [28]:
import SoccerNet

from SoccerNet.Downloader import SoccerNetDownloader
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="path/to/SoccerNet")
psswrd = "s0cc3rn3t"
mySoccerNetDownloader.password = psswrd

En el contexto de SoccerNet, los splits ("train", "valid", "test", "challenge") tienen los siguientes significados:

- train: Datos de entrenamiento estándar para modelos, con anotaciones completas.
- valid: Datos de validación para evaluar el rendimiento durante el desarrollo.
- test: Datos de prueba con etiquetas no públicas (normalmente usados en competiciones).
- challenge: Un subconjunto de datos específicos para la competición oficial o retos específicos, como el de Multi-View Foul Recognition.

Nos quedamos con __mvfouls__. Además añadimos el argumento _versión_ donde estableceremos que queremos data en 720p.

Con el lanzamiento del siguiente código se realizará la descarga, no lo ejecute a no ser que desee realizarla.

In [29]:
# Descarga los videos del Challenge Dataset en múltiples resoluciones (720p y 224p)
# Esto incluye todas las vistas necesarias (1 y 2) para las particiones relevantes
mySoccerNetDownloader.downloadDataTask(task="mvfouls", split=["train","valid","test","challenge"], password = psswrd, version = "720p")

__IMPORTANTE:__ El dataset se descargará en archivos zip, se deberán descomprimir.

El dataset consiste de 3.901 acciones. Cada acción está compuesta por al menos dos vídeos que muestra la acción en vivo y al menos una repetición.

El dataset se divide en:
- Training set (2916 actions).
- Validation set (411 actions).
- Test set (301 actions).
- Challenge set (273 actions without the annotations)

A continuación utilizaremos _OpenCV_ para comprobar que los vídeos y etiquetas se hayan descargado correctamente.

In [4]:
import cv2
import os

# Ruta al dataset
train_dataset_path = "path/to/SoccerNet/mvfouls/train_720p"

# Ejemplo de videos del set de entrenamiento
training_videos_path = os.path.join(train_dataset_path)
example_video = os.path.join(training_videos_path, "action_1/clip_0.mp4")

# Función para visualizar un video
def visualize_video(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    if not cap.isOpened():
        print(f"No se pudo abrir el video: {video_path}")
        return

    while cap.isOpened() and frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            print("Fin del video o error al leer el frame.")
            break
        
        cv2.imshow('Video Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):  # Presiona 'q' para salir
            break
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

# Visualiza el video de ejemplo
visualize_video(example_video)


También verificaremos que las etiquetas se encuentran en el formato correcto, y que se asocian correctamente con los vídeos.

In [21]:
import json

# Ruta a las etiquetas del training set
labels_path = os.path.join(train_dataset_path, "annotations.json")

# Carga las etiquetas
with open(labels_path, "r") as f:
    data = json.load(f)

print(f"Total de acciones etiquetadas: {data['Number of actions']}")
for action_index, action_data in data['Actions'].items():
    print(f"Índice de la acción: {action_index}")
    print(f"Detalles de la acción: {action_data}")
    break  # Probaremos solo la primera acción


Total de acciones etiquetadas: 2916
Índice de la acción: 0
Detalles de la acción: {'UrlLocal': 'england_epl\\2014-2015\\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley', 'Offence': 'Offence', 'Contact': 'With contact', 'Bodypart': 'Upper body', 'Upper body part': 'Use of shoulder', 'Action class': 'Challenge', 'Severity': '1.0', 'Multiple fouls': '', 'Try to play': '', 'Touch ball': '', 'Handball': 'No handball', 'Handball offence': '', 'Clips': [{'Url': 'Dataset/Train/action_0/clip_0', 'Camera type': 'Main camera center', 'Timestamp': 1730826, 'Replay speed': 1.0}, {'Url': 'Dataset/Train/action_0/clip_1', 'Camera type': 'Close-up player or field referee', 'Timestamp': 1744173, 'Replay speed': 1.8}]}


A continuación extraeremos la información de cada clip.

In [6]:
actions = data['Actions']

for action_id, action in actions.items():
    print(f"Acción número {action_id}:")
    print(f"  > Ofensa: {action['Offence']}")
    print(f"  > Parte del cuerpo: {action['Bodypart']}")
    print(f"  > Tipo de acción: {action['Action class']}")
    print(f"  > Severidad: {action['Severity']}")
    
    for clip in action['Clips']:
        clip_url = clip['Url']
        camera_type = clip['Camera type']
        timestamp = clip['Timestamp']
        print(f"    - Clip - {camera_type}: {clip_url} @ {timestamp}ms")


Acción número 0:
  > Ofensa: Offence
  > Parte del cuerpo: Upper body
  > Tipo de acción: Challenge
  > Severidad: 1.0
    - Clip - Main camera center: Dataset/Train/action_0/clip_0 @ 1730826ms
    - Clip - Close-up player or field referee: Dataset/Train/action_0/clip_1 @ 1744173ms
Acción número 1:
  > Ofensa: Offence
  > Parte del cuerpo: Under body
  > Tipo de acción: Tackling
  > Severidad: 3.0
    - Clip - Main camera center: Dataset/Train/action_1/clip_0 @ 2400217ms
    - Clip - Close-up player or field referee: Dataset/Train/action_1/clip_1 @ 2415695ms
Acción número 2:
  > Ofensa: Offence
  > Parte del cuerpo: Under body
  > Tipo de acción: Standing tackling
  > Severidad: 3.0
    - Clip - Main camera center: Dataset/Train/action_2/clip_0 @ 206821ms
    - Clip - Close-up player or field referee: Dataset/Train/action_2/clip_1 @ 230429ms
    - Clip - Close-up player or field referee: Dataset/Train/action_2/clip_2 @ 238257ms
Acción número 3:
  > Ofensa: Offence
  > Parte del cuerpo:

Ahora visualizaremos las repeticiones de una misma acción a la vez.

In [22]:
import cv2
import os

# Ruta al dataset
train_dataset_path = "path/to/SoccerNet/mvfouls/train_720p"

# Ruta de los dos clips
clip_0_path = os.path.join(train_dataset_path, "action_1/clip_0.mp4")
clip_1_path = os.path.join(train_dataset_path, "action_1/clip_1.mp4")


def visualize_clips_synchronized(clip_1_path, clip_2_path, timestamp_1=0, timestamp_2=0, num_frames=100):
    # Carga ambos videos
    cap1 = cv2.VideoCapture(clip_1_path)
    cap2 = cv2.VideoCapture(clip_2_path)
    
    if not cap1.isOpened() or not cap2.isOpened():
        print(f"Error al abrir los videos: {clip_1_path}, {clip_2_path}")
        return

    # Calcula los FPS de ambos videos
    fps1 = cap1.get(cv2.CAP_PROP_FPS)
    fps2 = cap2.get(cv2.CAP_PROP_FPS)

    # Calcula el fotograma inicial para cada video basado en los timestamps
    start_frame_1 = int(timestamp_1 / 1000 * fps1)
    start_frame_2 = int(timestamp_2 / 1000 * fps2)

    # Posiciona los videos en los fotogramas iniciales
    cap1.set(cv2.CAP_PROP_POS_FRAMES, start_frame_1)
    cap2.set(cv2.CAP_PROP_POS_FRAMES, start_frame_2)

    frame_count = 0
    while frame_count < num_frames:
        # Lee un frame de cada video
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        # Si algún video termina, salimos del loop
        if not ret1 or not ret2:
            print("Fin de uno de los videos o error al leer el frame.")
            break

        # Muestra los frames en ventanas separadas
        combined_frame = cv2.hconcat([frame1, frame2])
        cv2.imshow('Clips Sincronizados', combined_frame)

        # Sincroniza la reproducción (espera entre frames)
        if cv2.waitKey(int(1000 / max(fps1, fps2))) & 0xFF == ord('q'):  # Presiona 'q' para salir
            break

        frame_count += 1

    # Libera los recursos y cierra las ventanas
    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

# Sincroniza y visualiza los clips de ejemplo
# Supongamos que los timestamps están en milisegundos
visualize_clips_synchronized(clip_0_path, clip_1_path, timestamp_1=0, timestamp_2=1000)